# EnsLSTM
### Importing libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

from keras.optimizers import Adam
from keras.layers import Dropout
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, concatenate

### Loading the train dataset

In [2]:
water_consumption = pd.read_csv("train.csv")
water_consumption.head(10)

,Year,Month,Consumer_type,Consumption,Consumer_number,Installation_zone
0,2013,1,domestic,0,MOGV36480546611521,Installation_zone 1
1,2013,1,industrial,5,BECS02817768252637,Installation_zone 2
2,2013,1,domestic,6,VRFW65577141436242,Installation_zone 2
3,2013,1,domestic,1,QLLI18662653137621,Installation_zone 2
4,2013,1,domestic,13,HYUO61823402850645,Installation_zone 2
5,2013,1,industrial,27,FHMG62751338090488,Installation_zone 2
6,2013,1,industrial,5,APVF78863215212358,Installation_zone 2
7,2013,1,domestic,31,MXWL75757930683403,Installation_zone 2
8,2013,1,industrial,2,NVMY31359391120094,Installation_zone 2
9,2013,1,industrial,0,PZAN37359795617576,Installation_zone 2


In [3]:
train, test = train_test_split(water_consumption, test_size=0.2,
                               stratify=water_consumption['Consumer_type'], random_state=42)

## Model

In [4]:
class EnsLSTM():
  def __init__(self):
    self.networks = {}

  def fit(self, water_consumption):
    installation_groups = self.create_time_matrix_by_groups(water_consumption)

    print("Training ...")
    for group_name, group in installation_groups:
      print(group_name)
      X_data, y_data = self.preprocess_data(group,
                                     target_column='Consumer_type',
                                     feature_column='Consumption')

      net = self.createIndLSTM(X_data.shape, len(group['Consumer_type'].unique()))
      net.fit(X_data, y_data, epochs=10, batch_size=32, validation_split=0.2)

      self.networks[group_name] = net

  def create_time_matrix_by_groups(self, water_consumption):
    print("Creating temporal matrix ...")

    water_consumption_months = self.agg_months(water_consumption,
     ['Consumer_type', 'Consumer_number', 'Installation_zone', 'Year'])

    water_consumption_years = self.agg_years(water_consumption_months,
     ['Consumer_type', 'Consumer_number', 'Installation_zone'])

    installation_groups = water_consumption_years.groupby('Installation_zone')

    return installation_groups

  def agg_months(self, water_consumption, group_features):
    all_months = list(range(1, 13))
    water_consumption_months = water_consumption.groupby(group_features).apply(
        lambda group: pd.Series({
            'Consumption': [group.loc[group['Month'] == month, 'Consumption'].values[0] if month in group['Month'].values else 0 for month in all_months]
        })
    ).reset_index()

    return water_consumption_months

  def agg_years(self, water_consumption_months, group_features):
    unique_years = water_consumption_months['Year'].unique()

    water_consumption_years = water_consumption_months.groupby(group_features).apply(
        lambda group: pd.Series({
            'Consumption': [group.loc[group['Year'] == year, 'Consumption'].values[0] if year in group['Year'].values else np.zeros(12) for year in unique_years]
        })
    ).reset_index()

    return water_consumption_years

  def preprocess_data(self, df, target_column=None, feature_column='Consumption', test_size=0.2, random_state=42):
    X = np.array(df[feature_column].tolist())
    X = np.transpose(X, (0, 2, 1))

    if target_column == None:
      return X

    y = pd.get_dummies(df[target_column]).values

    return X, y

  def createIndLSTM(self, shape, num_labels):
    net = Sequential()
    net.add(LSTM(50, input_shape=(shape[1], shape[2])))
    net.add(Dense(num_labels, activation='softmax'))

    if num_labels > 1:
      net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
      net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return net

  def eval(self, water_consumption):
    installation_groups = self.create_time_matrix_by_groups(water_consumption)
    print("Test ...")

    right = 0
    total = 0
    for group_name, group in installation_groups:
      print(group_name)
      X_data, y_data = self.preprocess_data(group,
                                     target_column='Consumer_type',
                                     feature_column='Consumption')


      y_pred_proba = self.networks[group_name].predict(X_data)

      y_data = np.argmax(y_data, axis=1)
      y_pred = np.argmax(y_pred_proba, axis=1)

      tmp_right = np.sum(y_data == y_pred)
      right += tmp_right
      total += len(y_data)

    return right / total

#### Train

In [5]:
network = EnsLSTM()
network.fit(train)

Creating temporal matrix ...
Training ...
Installation_zone 1
Epoch 1/10
141/141 [==============================] - 4s 14ms/step - loss: 0.2134 - accuracy: 0.9637 - val_loss: 2.4326 - val_accuracy: 0.6643
Epoch 2/10
141/141 [==============================] - 1s 9ms/step - loss: 0.0359 - accuracy: 0.9942 - val_loss: 2.8395 - val_accuracy: 0.6643
Epoch 3/10
141/141 [==============================] - 1s 9ms/step - loss: 0.0350 - accuracy: 0.9942 - val_loss: 2.7814 - val_accuracy: 0.6643
Epoch 4/10
141/141 [==============================] - 1s 9ms/step - loss: 0.0330 - accuracy: 0.9942 - val_loss: 3.0642 - val_accuracy: 0.6643
Epoch 5/10
141/141 [==============================] - 1s 9ms/step - loss: 0.0321 - accuracy: 0.9942 - val_loss: 2.9646 - val_accuracy: 0.6643
Epoch 6/10
141/141 [==============================] - 1s 8ms/step - loss: 0.0309 - accuracy: 0.9942 - val_loss: 2.9489 - val_accuracy: 0.6643
Epoch 7/10
141/141 [==============================] - 2s 12ms/step - loss: 0.0322 - a

#### Test

In [6]:
acc = network.eval(test)

print("Acc: " + str(acc))

Creating temporal matrix ...
Test ...
Installation_zone 1
146/146 [==============================] - 1s 5ms/step
Installation_zone 10
11/11 [==============================] - 1s 6ms/step
Installation_zone 11
3/3 [==============================] - 1s 7ms/step
Installation_zone 12
4/4 [==============================] - 1s 5ms/step
Installation_zone 13
5/5 [==============================] - 0s 4ms/step
Installation_zone 14
3/3 [==============================] - 0s 6ms/step
Installation_zone 15
1/1 [==============================] - 0s 473ms/step
Installation_zone 16


8/8 [==============================] - 0s 4ms/step
Installation_zone 17
6/6 [==============================] - 0s 4ms/step
Installation_zone 18
4/4 [==============================] - 0s 5ms/step
Installation_zone 19
5/5 [==============================] - 0s 4ms/step
Installation_zone 2
102/102 [==============================] - 1s 3ms/step
Installation_zone 20
2/2 [==============================] - 0s 8ms/step
Installation_zone 21
4/4 [==============================] - 0s 5ms/step
Installation_zone 22
6/6 [==============================] - 0s 4ms/step
Installation_zone 23
5/5 [==============================] - 0s 4ms/step
Installation_zone 24
9/9 [==============================] - 0s 4ms/step
Installation_zone 25
6/6 [==============================] - 0s 4ms/step
Installation_zone 26
4/4 [==============================] - 1s 4ms/step
Installation_zone 27
3/3 [==============================] - 0s 6ms/step
Installation_zone 28
3/3 [==============================] - 1s 8ms/step
Installati

129/129 [==============================] - 1s 5ms/step
Installation_zone 30
2/2 [==============================] - 1s 6ms/step
Installation_zone 31
5/5 [==============================] - 0s 4ms/step
Installation_zone 32
4/4 [==============================] - 0s 5ms/step
Installation_zone 33
3/3 [==============================] - 0s 6ms/step
Installation_zone 34
3/3 [==============================] - 0s 5ms/step
Installation_zone 35
6/6 [==============================] - 0s 4ms/step
Installation_zone 36
5/5 [==============================] - 0s 4ms/step
Installation_zone 37
3/3 [==============================] - 0s 5ms/step
Installation_zone 38
5/5 [==============================] - 0s 5ms/step
Installation_zone 39
4/4 [==============================] - 0s 5ms/step
Installation_zone 4
131/131 [==============================] - 1s 4ms/step
Installation_zone 40
3/3 [==============================] - 0s 6ms/step
Installation_zone 41
3/3 [==============================] - 0s 6ms/step
Instal